In [12]:
import numpy as np
import pandas as pd 

from scrapy.selector import Selector
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import requests
import re

import datetime
from datetime import date
from time import sleep

In [2]:
# get the url's of the businesses featured in 'best of Melbourne' collection 
def get_business_urls(html):
    
    # restaurant url's
    try:
        url_list=Selector(text=html).xpath('//div[@class="row col-res-list collection_listings_container"]/div/div/div/a/@href').extract()
    except:
        url_list=np.NaN
        return np.NaN
    
    print(url_list)
    return url_list

In [29]:
# get the restaurant_id
def get_restaurant_id(html):

    # restaurant id
    try:
        restaurant_id = Selector(text=html).xpath('//div[@id="reviews-container"]/*//div[@data-snippet="restaurant-review"]/@data-res_id').extract_first()
    except:
        restaurant_id = np.NaN
        print('error on restaurant_id for : ', restaurant_id)
                   
    return restaurant_id

In [9]:
# start of code for getting the business url's in the 'best of Melbourne' collection

# open business reviews url
main_url = 'https://www.zomato.com/melbourne/great-food-no-bull'
main_url1 = "http://www.google.com"
driver = webdriver.Chrome(executable_path="chromedriver")
try:
    sleep(5)
    driver.get(main_url)
except:
    print('error in loading driver for business url: ', main_url)        
assert "Zomato" in driver.title

# parse first results page
sleep(5)
# set HTML to the response from the http request
html = driver.page_source

url_list = get_business_urls(html)

driver.close()

['https://www.zomato.com/melbourne/fujitei-st-kilda?zrp_bid=170919&zrp_pid=14&zrp_cid=284987', 'https://www.zomato.com/melbourne/larks-of-joy-northcote?zrp_bid=236561&zrp_pid=14&zrp_cid=326572', 'https://www.zomato.com/melbourne/48h-pizza-gnocchi-bar-south-yarra?zrp_bid=261400&zrp_pid=14&zrp_cid=340022', 'https://www.zomato.com/melbourne/guhng-mckillop-street-cbd-melbourne?zrp_bid=232257&zrp_pid=14&zrp_cid=324174', 'https://www.zomato.com/melbourne/prospect-espresso-camberwell?zrp_bid=224105&zrp_pid=14&zrp_cid=319177', 'https://www.zomato.com/melbourne/small-print-pizza-bar-windsor?zrp_bid=252835&zrp_pid=14&zrp_cid=335743', 'https://www.zomato.com/melbourne/the-hub-3070-northcote?zrp_bid=190835&zrp_pid=14&zrp_cid=297670', 'https://www.zomato.com/melbourne/railway-club-hotel-port-melbourne-port-melbourne?zrp_bid=231705&zrp_pid=14&zrp_cid=323809', 'https://www.zomato.com/melbourne/zero-gradi-brunswick?zrp_bid=198290&zrp_pid=14&zrp_cid=302999', 'https://www.zomato.com/melbourne/cucina-co-

In [30]:
# start of code for getting the reviews for each business featured in 'best of Melbourne' collection

# target count of reviews to extract
restaurant_id = []    

total_count=len(url_list)
# iterate through the url's of the businesses to extract the reviews
for business_url in url_list:

    # open business reviews url
    driver = webdriver.Chrome(executable_path="chromedriver")
    try:
        driver.get(business_url + '/reviews')
    except:
        print('error in loading driver for business url: ', business_url)
        
    assert "Zomato" in driver.title

    restaurant = get_restaurant_id(html)

    restaurant_id.append(restaurant)
    
    
    # close the Chrome webdriver
    driver.close()

KeyboardInterrupt: 

In [31]:
restaurant_id

[None, None]

In [32]:
# get the reviews for a specific business
def get_reviews_per_url(html):

    reviews_per_business = []
    review_index = 0
    
    # get restaurant id and review id (separately, as these two are outside the xpath scope defined below)
    # review id list 
    try:
        review_id_list = Selector(text=html).xpath('//div[@id="reviews-container"]/*//div[@data-snippet="restaurant-review"]/@data-review_id').extract()
        print('# of review ids: ',len(review_id_list))
    except:
        review_id_list= np.NaN
        print('error on review id list for : ', review_id_list)
        
    # restaurant id
    try:
        restaurant_id = Selector(text=html).xpath('//div[@id="reviews-container"]/*//div[@data-snippet="restaurant-review"]/@data-res_id').extract()[0]
    except:
        restaurant_id = np.NaN
        print('error on restaurant_id for : ', restaurant_id)
        
    # get data for displayed reviews 
    visible_reviews_full = Selector(text=html).xpath('//div[@id="reviews-container"]/*//div[@data-snippet="restaurant-review"]/div[contains(@class,"ui segment")]').extract()
    num_reviews=len(visible_reviews_full)

    # iterate through the displayed reviews
    for review in visible_reviews_full:

        # review id
        try:
            review_id= review_id_list[review_index]
        except:
            print('error on iterating through review id for: ', review_id_list)
            
        # user id
        try:
            user_id=Selector(text=review).xpath('//div[contains(@class,"content col-l-11")]/*/a[contains(@href,"https://www.zomato.com")]/@data-entity_id').extract()[0]
        except:
            user_id=np.NaN

        # user name
        try:
            user_name=Selector(text=review).xpath('//div[contains(@class,"content col-l-11")]/*/a[contains(@href,"https://www.zomato.com")]/text()').extract()[0].strip()
        except:
            user_name=np.NaN

        # no. of reviews by user
        try:
            user_review_count=Selector(text=review).xpath('//div[contains(@class,"content col-l-11")]/span[contains(@class,"grey-text")]/text()').extract()[0].strip().split(',')[0].strip().split(' ')[0]
        except:
            user_review_count=np.NaN

        # no. of followers of user
        try:
            user_followers_count=Selector(text=review).xpath('//div[contains(@class,"content col-l-11")]/span[contains(@class,"grey-text")]/text()').extract()[0].strip().split(',')[1].strip().split(' ')[0]
        except:
            user_followers_count=np.NaN

        # review date
        try:
            review_date=Selector(text=review).xpath('//time/@datetime').extract()[0]
        except:
            review_date=np.NaN

        # star rating
        try:
            star_rating=Selector(text=review).xpath('//div[contains(@class,"rev-text")]/div[contains(@class,"ttupper fs12px")]/@aria-label').extract()[0].split(' ')[1]
        except:
            star_rating=np.NaN

        # textual review
        try:
            textual_review=Selector(text=review).xpath('//div[contains(@class,"rev-text ")]/text()').extract()[1:]
        except:
            textual_review=np.NaN

        # clean textual review, convert to string
        try:
            textual_review[0] = textual_review[0].replace('\xa0\n', '').strip()
            textual_review = ''.join([line.strip() for line in textual_review if line.strip() is not ''])
        except:
            print('error in cleaning textual_review for : ', textual_review)
            
        # append scraped review to list
        reviews_per_business.append([review_id, restaurant_id, user_id, user_name, user_review_count, user_followers_count, review_date, star_rating, textual_review])
        review_index += 1
        
    return reviews_per_business, num_reviews

In [37]:
# start of code for getting the reviews for each business featured in 'best of Melbourne' collection

# target count of reviews to extract
review_target_count = 200
all_business_reviews = []
num_reviews_list = []
business_with_zero_reviews = []

monitor_count=0    

total_count=len(url_list)
# iterate through the url's of the businesses to extract the reviews
for business_url in url_list:

    # open business reviews url
    driver = webdriver.Chrome(executable_path="chromedriver")
    try:
        driver.get(business_url + '/reviews')
    except:
        print('error in loading driver for business url: ', business_url)
        
    assert "Zomato" in driver.title

    # if there's a Popular (Reviews) section, change and click on All Reviews section
    driver = switch_from_Popular_to_All_Reviews(driver, business_url + '/reviews')
    
    # display more reviews (in preparation for scraping the target # of reviews)
    driver, visible_review_count = display_more_reviews(driver, review_target_count)

    # get final count of visible reviews (for this particular business)
    html = driver.page_source

    # if review list is not empty
    if visible_review_count > 0:
        # scrape the displayed reviews
        reviews_per_business, num_reviews = get_reviews_per_url(html)

        num_reviews_list.append(num_reviews)
        
        # add reviews per business to overall list of business reviews
        all_business_reviews += reviews_per_business

    # businesses with zero reviews
    else:
        business_with_zero_reviews.append(business_url)

    print('url: ', business_url)    
    monitor_count += 1
    print(str(monitor_count), ' of ', str(total_count), " businesses completed...")

    # save reviews for every 20 businesses to a csv file
    if (monitor_count % 20) == 0:
        reviews_20 = pd.DataFrame(all_business_reviews)
        # save the latest 20 businesses
        reviews_20.to_csv('./datasets/Reviews - Best of Melbourne ' + str(monitor_count-20) + ' to ' + str(monitor_count-1) +  '.csv')
        # clear business reviews list to make way for the next 20 businesses
        all_business_reviews = []
    
    # close the Chrome webdriver
    driver.close()
    
print('business with zero reviews: ', business_with_zero_reviews)
print('number of reviews: ', num_reviews_list)

Popular Reviews section exists for: https://www.zomato.com/melbourne/fujitei-st-kilda?zrp_bid=170919&zrp_pid=14&zrp_cid=284987/reviews
# of review ids:  26
url:  https://www.zomato.com/melbourne/fujitei-st-kilda?zrp_bid=170919&zrp_pid=14&zrp_cid=284987
1  of  284  businesses completed...
Popular Reviews section exists for: https://www.zomato.com/melbourne/larks-of-joy-northcote?zrp_bid=236561&zrp_pid=14&zrp_cid=326572/reviews
# of review ids:  8
url:  https://www.zomato.com/melbourne/larks-of-joy-northcote?zrp_bid=236561&zrp_pid=14&zrp_cid=326572
2  of  284  businesses completed...
Popular Reviews section exists for: https://www.zomato.com/melbourne/48h-pizza-gnocchi-bar-south-yarra?zrp_bid=261400&zrp_pid=14&zrp_cid=340022/reviews
# of review ids:  175
url:  https://www.zomato.com/melbourne/48h-pizza-gnocchi-bar-south-yarra?zrp_bid=261400&zrp_pid=14&zrp_cid=340022
3  of  284  businesses completed...
Popular Reviews section exists for: https://www.zomato.com/melbourne/guhng-mckillop-str

FileNotFoundError: [Errno 2] No such file or directory: './datasets/Reviews - Best of Melbourne 0 to 19.csv'

In [35]:
# display more reviews by clicking on 'Load More' element
def display_more_reviews(driver, review_target_count):
    
    html = driver.page_source
    
    try:
        # get the visible reviews and review count (for this business)
        visible_reviews = Selector(text=html).xpath('//div[@id="reviews-container"]/*//div[@data-snippet="restaurant-review"]/div[contains(@class,"ui segment")]').extract()    
        visible_review_count = len(visible_reviews)

        # open more reviews by clicking on Load More until the desired # of reviews (for this particular business) is displayed
        while visible_review_count < review_target_count:
            try:
                # access and click on Load More element - to extend the list of displayed reviews
                load_more_element = driver.find_element_by_xpath('//div[@class="load-more bold ttupper tac cursor-pointer fontsize2"]/span[contains(@class,"zred")]')
                load_more_element.click()
                sleep(5)

            except:
                break

            # update the count of visible reviews (for this particular business)
            html = driver.page_source
            visible_reviews = Selector(text=html).xpath('//div[@id="reviews-container"]/*//div[@data-snippet="restaurant-review"]/div[contains(@class,"ui segment")]').extract()
            visible_review_count = len(visible_reviews)
    except:
        print('ERROR IN DISPLAY_MORE_REVIEWS')
    
    return driver, visible_review_count

In [36]:
# Switch from Popular Reviews to All Reviews, if applicable
def switch_from_Popular_to_All_Reviews(driver, url):
    
    html = driver.page_source
    try:
        # see if Popular Reviews tab exists
        popular_section = Selector(text=html).xpath('//div[@id="selectors"]/a[contains(@data-sort,"reviews-top")]/text()').extract()[0]
        try:
            # access and click on All Reviews element - to switch from Popular Reviews list
            load_more_element = driver.find_element_by_xpath('//div[@id="selectors"]/a[contains(@data-sort,"reviews-dd")]')
            sleep(3)
            load_more_element.click()
            sleep(1)
            print('Popular Reviews section exists for: ' + url)

        except:
            display('All Reviews element not found, or click not successful for:' + url)
    
    except:
        print('No Popular Reviews section for: ' + url)
        
    return driver

In [39]:
 reviews_per_business

[['41243179',
  '16572701',
  '110174',
  'SDeans',
  '139',
  '179',
  '2019-01-26 12:56:42',
  '3.0',
  "Tried an egg & cheese and an egg & onion galettes and a sugar with lemon juice crepe after. The galettes were just about okay, a bit too much cheese in one, and the crepe was ruined as it was drenched in lemon juice which made it sour and soaked. Disappointed that they didn't even offer to replace it.There is a much better creperie in Fitzroy which I would recommend over this place.Service is friendly, but the food needs to be good for a place to get highly rated."],
 ['40941754',
  '16572701',
  '51624882',
  '刘湃',
  '2',
  '8',
  '2019-01-06 16:32:45',
  '4.0',
  'Very delicious :) I like the sausage one which has pasta source inside:) however, the environment is not so so so good, it can be accepted. I will visit there again!'],
 ['40790704',
  '16572701',
  '44284877',
  'Crabapples For Lunch',
  '214',
  '51',
  '2018-12-28 06:56:31',
  '4.0',
  "As far as crepe places go thi